# 看穿你的意圖(cntk版)

In [1]:
import codecs
import numpy as np
import random
import pickle
import os
import math

In [2]:
intent_data=codecs.open('intent_list.txt',mode='r',encoding='utf-8-sig').readlines()
intent_dict={}
for i in range(len(intent_data)):
    intent_dict[i]=intent_data[i].strip() 
intent_dict

{0: '打開app應用',
 1: '詢問公車路線',
 2: '數學計算',
 3: '閒聊',
 4: '詢問電影場次',
 5: '查詢聯絡人',
 6: '查詢食譜',
 7: '詢問日期',
 8: '電子郵件指令',
 9: '詢問電視節目',
 10: '詢問航班',
 11: '詢問健康資訊',
 12: '詢問樂透',
 13: '詢問地圖',
 14: '詢問足球賽事',
 15: '簡訊指令',
 16: '音樂指令',
 17: '詢問新聞',
 18: '詢問小說',
 19: '詢問詩詞',
 20: '詢問廣播節目',
 21: '詢問謎語',
 22: '鬧鐘時程設定',
 23: '詢問股票',
 24: '電話指令',
 25: '詢問火車班次',
 26: '詢問翻譯',
 27: '切換電視頻道',
 28: '影片播放指令',
 29: '詢問天氣',
 30: '開啟網站'}

In [3]:
train_data=codecs.open('../Data/ex13_train/Intent_TrainData.txt',mode='r',encoding='utf-8-sig').readlines()
maxlen=max([len(s) for s in train_data])
print("最大長度為:{0}".format(maxlen))
print(len(train_data))
print(train_data[:10])

最大長度為:52
2299
['打开熊猫看书\t0\r\n', 'opera浏览器\t0\r\n', '钛备份\t0\r\n', '打开sky电话\t0\r\n', '帮我找菜单\t0\r\n', '找愤怒的小鸟\t0\r\n', '正韵打开相机\t0\r\n', '我把相机打开\t0\r\n', '上QQ我\t0\r\n', '打开uc浏览器v八\t0\r\n']


In [4]:
test_data=codecs.open('../Data/ex13_train/Intent_TestData.txt',mode='r',encoding='utf-8-sig').readlines()
maxlen=max([len(s) for s in test_data])
print("最大長度為:{0}".format(maxlen))
print(len(test_data))
print(test_data[:10])

最大長度為:69
1437
['打开我的浏览器\t0\r\n', '打开qq浏览器\t0\r\n', '打开音乐播放器\t0\r\n', '启动浏览器\t0\r\n', '他开酷狗\t0\r\n', '帮我搜索三国杀\t0\r\n', '打开极品飞车\t0\r\n', '公交查询\t0\r\n', '打开搜狐新闻\t0\r\n', '图库\t0\r\n']


In [5]:
chars={}
with open('hanzi2vec256.pkl','rb') as fp:
    chars = pickle.load(fp)

In [6]:
random.shuffle(train_data) #把訓練數據集順序打亂
random.shuffle(test_data) #把訓練數據集順序打亂
idx0=0
idx1=0

def data_augumentation(txt):
    txt=list(txt)
    if random.randint(0,10)<=3 and len(txt)>6:#隨機對調字的順序
        pos= random.randint(1,len(txt)-2)
        w1=txt[pos]
        w2=txt[pos+1]
        txt[pos]=w2
        txt[pos+1]=w1
    if random.randint(0,100)%7<2 and len(txt)>=6:
        wordlist=['的','了','若','可','然','也','或','，','。',' ','  ']
        w1=random.choice(wordlist)
        txt.insert(random.randint(1,len(txt)-2), w1)
    if random.randint(0,100)%5<2:
        wordlist=['，','。','？','！',' ','  ']
        w1=random.choice(wordlist)
        txt.append(w1)
    return ''.join(txt)

def get_next_minibatch(minibatch_size=16,is_train=True):
    global idx0,idx1

    features = []
    labels = []
    groundtruths = []
    while len(features) < minibatch_size:
        try:
            items=[]
            if is_train:
                items=train_data[idx0].strip().split('\t')
            else:
                items=test_data[idx1].strip().split('\t')
            txt=items[0]
            new_txt=items[0]
            if is_train:
                new_txt=data_augumentation(new_txt)
            lab=int(items[1])
            feature=np.zeros((256,128)) #預設都為零
            label=np.zeros(31)
            label[lab]=1
            for i in range(128):
                if i< len(new_txt):
                    feature[:,i]=chars[new_txt[i].lower()] if new_txt[i].lower() in chars else np.random.uniform(-1,1,256)
                    #替換為字向量，若是沒有這個字，填隨機亂數
            features.append(feature)
            labels.append(label)
            groundtruths.append(list(txt))
            if is_train:
                idx0 += 1
                if idx0 >= len(train_data):
                    idx0 = 0
                    random.shuffle(train_data)
            else:
                idx1 += 1
                if idx1 >= len(test_data):
                    idx1 = 0
                    random.shuffle(test_data)
        except Exception as e:
            print(e)
    return np.asarray(features).astype(np.float32), np.asarray(labels).astype(np.float32), groundtruths


features,labels,groundtruths=get_next_minibatch(minibatch_size=3)
print(features)
print(labels)
print(groundtruths)

[[[-0.63979566 -0.2624278  -0.40248755 ...  0.          0.
    0.        ]
  [-0.9186666   0.1217322   0.8014214  ...  0.          0.
    0.        ]
  [ 1.205029   -0.14789534  0.05057659 ...  0.          0.
    0.        ]
  ...
  [ 1.162124    0.515679    0.03682213 ...  0.          0.
    0.        ]
  [ 0.6169786  -0.62525386 -0.61422324 ...  0.          0.
    0.        ]
  [-0.2580631  -0.4071354   0.05600768 ...  0.          0.
    0.        ]]

 [[ 0.07990474 -0.9881383  -1.1837695  ...  0.          0.
    0.        ]
  [ 0.6135107   0.20621426 -0.2804067  ...  0.          0.
    0.        ]
  [-0.49800426 -0.2703897   0.11371613 ...  0.          0.
    0.        ]
  ...
  [ 0.4830793   0.7456922   0.2825017  ...  0.          0.
    0.        ]
  [-0.18418962 -0.5534653   0.8914223  ...  0.          0.
    0.        ]
  [-0.04748059 -1.001024   -0.8588255  ...  0.          0.
    0.        ]]

 [[ 0.3429284  -0.56450915 -0.2175991  ...  0.          0.
    0.        ]
  [ 0.741

In [7]:
from cntk.initializer import *
from cntk.layers import *
from cntk.layers.models.attention import *
from cntk.layers.typing import *
from cntk.learners import adam, momentum_as_time_constant_schedule, learning_rate_schedule, UnitType
from cntk.logging import log_number_of_parameters, ProgressPrinter
from cntk.losses import *
from cntk.metrics import classification_error
from cntk.ops import *
from cntk.train import Trainer
from cntk.device import try_set_default_device, cpu, gpu

# 是否使用GPU
is_gpu = True

if is_gpu:
    try_set_default_device(gpu(0))
else:
    try_set_default_device(cpu())

![md_images](../Images/textcnn.jpg)

In [8]:

#為了解決類別間不均衡的焦距損失
def focal_loss(output, target, gamma=2, axis=-1):
    return negate(reduce_sum(target * C.pow(1 - output, gamma) * log(output), axis))



def create_model(input):
    with default_options(init=he_normal(0.02), activation=leaky_relu, max_temp_mem_size_in_samples=1):
        conv_3 = Convolution1D(3, 64,strides=2, pad=True, reduction_rank=1, activation=leaky_relu,name='conv_3')(input)
        conv_3 = BatchNormalization(map_rank=1)(conv_3)
      

        conv_5 = Convolution1D(5, 64, strides=2,pad=True, reduction_rank=1, activation=leaky_relu,name='conv_5')(input)
        conv_5 = BatchNormalization(map_rank=1)(conv_5)
    

        conv_7 = Convolution1D(7,64,strides=2, pad=True, reduction_rank=1, activation=leaky_relu,name='conv_7')(input)
        conv_7 = BatchNormalization(map_rank=1)(conv_7)
    

        merge = splice(conv_3, conv_5, conv_7, axis=0)
        drop1=Dropout(0.5)(merge)
        conv_3d = Convolution1D(3, 64, pad=True, activation=leaky_relu )(drop1)
        conv_1d = Convolution1D(1, 16, pad=True, activation=leaky_relu )(conv_3d)
        all = reshape(conv_1d, -1)
        all = Dense(31, activation=softmax)(all)
        return all

In [ ]:
learning_rate = 0.0005
num_epochs = 300
minibatch_size = 32


# 訓練意圖識別模型
def train():
    global learning_rate, minibatch_size

    # 定義輸入變數
    input_var = input_variable((256,128))
    label_var = input_variable((31))

    # 定義模型輸出
    z = create_model(input_var)
    if os.path.exists("Models/textcnn.cnn"):
        model = Function.load("Models/textcnn.cnn")
        z = model(input_var)
        print('載入歷史模型，繼續訓練')

    # 定義進行訓練的損失函數以及錯誤率計算

    loss = cross_entropy_with_softmax(z, label_var)+focal_loss(z, label_var)
    errs = classification_error(z, label_var)

    # 列印模型參數
    log_number_of_parameters(z);
    print()

    # 定義訓練器
    num_trained_samples_since_last_report = 0
    progress_printer = ProgressPrinter(freq=20, first=5, tag='Training', num_epochs=300)
    learner = adam(z.parameters,
                   lr=learning_rate_schedule([learning_rate], UnitType.sample, 300),
                   momentum=momentum_as_time_constant_schedule([minibatch_size / -math.log(0.95)], epoch_size=300),
                   l1_regularization_weight=0.001, l2_regularization_weight=5e-3,
                   gaussian_noise_injection_std_dev=0.0002,
                   gradient_clipping_threshold_per_sample=16.6)
    trainer = Trainer(z, (loss, errs), learner, progress_printer)

    for epoch in range(num_epochs):
        mbs = 0
        num_trained_samples = 0
        while mbs <= 1000:
            features, labels, truths = get_next_minibatch(minibatch_size)
            # 進行訓練
            trainer.train_minibatch({input_var: features, label_var: labels})
            if mbs%10 and mbs>0:
                z.save("Models/textcnn.cnn")
                z.save("Models/textcnn_{0}.cnn".format(epoch))
            if mbs%100==0 and mbs>0 :
                test_features, test_labels, test_truths = get_next_minibatch(minibatch_size,is_train=False)
                trainer.test_minibatch({input_var: test_features, label_var: test_labels})
                e=z(test_features)
                for i in range(5):
                    sentence=test_truths[i]
                    if len(sentence)>=3:
                        result=np.argmax(e[i])
                        label=np.argmax(test_labels[i])
                        print('{0}  ({1})=>({2})'.format(''.join(sentence),intent_dict[label],intent_dict[result]))

            mbs += 1
        # 回報每個epoch訓練進度以及相關指標
        trainer.summarize_training_progress()
        # 寫作測試


In [ ]:
 train()

載入歷史模型，繼續訓練
Training 316079 parameters in 18 parameter tensors.

Learning rate per 1 samples: 8e-05
 Minibatch[   1-   1]: loss = 2.958117 * 32, metric = 21.88% * 32;
 Minibatch[   2-   2]: loss = 2.821088 * 32, metric = 6.25% * 32;
 Minibatch[   3-   3]: loss = 2.741488 * 32, metric = 6.25% * 32;
 Minibatch[   4-   4]: loss = 2.772820 * 32, metric = 9.38% * 32;
 Minibatch[   5-   5]: loss = 2.705120 * 32, metric = 3.12% * 32;
 Minibatch[   6-  20]: loss = 2.822984 * 480, metric = 7.29% * 480;
 Minibatch[  21-  40]: loss = 2.833566 * 640, metric = 9.69% * 640;
 Minibatch[  41-  60]: loss = 2.908121 * 640, metric = 10.47% * 640;
 Minibatch[  61-  80]: loss = 2.811317 * 640, metric = 8.91% * 640;
 Minibatch[  81- 100]: loss = 2.803230 * 640, metric = 7.81% * 640;
查找科大讯飞股票的股价  (詢問股票)=>(詢問股票)
烟台到齐齐哈尔的火车票  (詢問火車班次)=>(詢問火車班次)
抬头见喜  (影片播放指令)=>(開啟網站)
怎么走爽了呢  (閒聊)=>(閒聊)
么么么哒  (閒聊)=>(閒聊)
 Minibatch[ 101- 120]: loss = 2.821710 * 640, metric = 8.12% * 640;
 Minibatch[ 121- 140]: loss = 2.834663 * 